# Setup Visdom

Install it with:

`pip install visdom`

Start the server:

`python -m visdom.server`

Visdom now can be accessed at http://localhost:8097 in the browser.


# LDA Training Visualization

To monitor the LDA training, a list of Metrics can be passed to the LDA function call for plotting their values live as the training progresses.  

Let's plot the training stats for an LDA model being trained on Lee corpus. We will use the four evaluation metrics available for topic models in gensim: Coherence, Perplexity, Topic diff and Convergence. (using separate hold_out and test corpus for evaluating the perplexity)

In [1]:
import os
import re
import gensim
from gensim.models import ldamodel
from gensim.corpora.dictionary import Dictionary

# Set file names for train data
test_data_dir = '{}'.format(os.sep).join([gensim.__path__[0], 'test', 'test_data'])
lee_corpus = test_data_dir + os.sep + 'lee.cor'

def read_corpus(fname):
    texts = []
    with open(fname, encoding="ISO-8859-1") as f:
        for line in f:
            # lower case all words
            lowered = line.lower()
            # remove punctuation and split into seperate words
            words = re.compile('\w+').findall(lowered)
            texts.append(words)
    return texts

texts = read_corpus(lee_corpus)

# Split test data into hold_out and test corpus
training_texts = texts[:25]
holdout_texts = texts[25:40]
test_texts = texts[40:50]

training_dictionary = Dictionary(training_texts)
holdout_dictionary = Dictionary(holdout_texts)
test_dictionary = Dictionary(test_texts)

training_corpus = [training_dictionary.doc2bow(text) for text in training_texts]
holdout_corpus = [holdout_dictionary.doc2bow(text) for text in holdout_texts]
test_corpus = [test_dictionary.doc2bow(text) for text in test_texts]

Using TensorFlow backend.


In [2]:
from gensim.models.callbacks import CoherenceMetric, DiffMetric, PerplexityMetric, ConvergenceMetric

# define perplexity callback for hold_out and test corpus
pl_holdout = PerplexityMetric(corpus=holdout_corpus, logger="visdom", viz_env="LdaModel", title="Perplexity (hold_out)")
pl_test = PerplexityMetric(corpus=test_corpus, logger="visdom", viz_env="LdaModel", title="Perplexity (test)")

# define other remaining metrics available
ch_umass = CoherenceMetric(corpus=training_corpus, coherence="u_mass", logger="visdom", viz_env="LdaModel", title="Coherence (u_mass)")
diff_kl = DiffMetric(distance="kullback_leibler", logger="visdom", viz_env="LdaModel", title="Diff (kullback_leibler)")
convergence_jc = ConvergenceMetric(distance="hellinger", logger="visdom", viz_env="LdaModel", title="Convergence (jaccard)")

callbacks = [pl_holdout, pl_test, ch_umass, diff_kl, convergence_jc]

# training LDA model
model = ldamodel.LdaModel(corpus=training_corpus, id2word=training_dictionary, passes=5, num_topics=5, callbacks=callbacks)

When the model is set for training, you can open http://localhost:8097 to see the training progress.

In [3]:
# to get a metric value on a trained model
print(CoherenceMetric(corpus=training_corpus, coherence="u_mass").get_value(model=model))

-22.4298221364


The four types of graphs which are plotted for LDA:

**Coherence**

Coherence is a measure used to evaluate topic models. A good model will generate coherent topics, i.e., topics         with high topic coherence scores. Good topics are topics that can be described by a short label based on the topic     terms they spit out. 

<img src="Coherence.gif">

Now, this graph along with the others explained below, can be used to decide if it's time to stop the training. We     can see if the value stops changing after some epochs and that we are able to get the highest possible coherence       of our model.  


**Perplexity**

Perplexity is a measurement of how well a probability distribution or probability model predicts a sample. In LDA, topics are described by a probability distribution over vocabulary words. So, perplexity can be used to compare probabilistic models like LDA.

<img src="Perplexity.gif">

For a good model, perplexity should be as low as possible.


**Topic Difference**

Topic Diff calculates the distance between two LDA models. This distance is calculated based on the topics, by either using their probability distribution over vocabulary words (kullback_leibler, hellinger) or by simply using the common vocabulary words between the topics from both model.

<img src="Diff.gif">

In the heatmap, X-axis define the Epoch no. and Y-axis define the distance between the identical topic from consecutive epochs. For ex. a particular cell in the heatmap with values (x=3, y=5, z=0.4) represent the distance(=0.4) between the topic 5 from 3rd epoch and topic 5 from 2nd epoch. With increasing epochs, the distance between the identical topics should decrease.
  
  
**Convergence**

Convergence is the sum of the difference between all the identical topics from two consecutive epochs. It is basically the sum of column values in the heatmap above.

<img src="Convergence.gif">

The model is said to be converged when the convergence value stops descending with increasing epochs.

# Training Logs

We can also log the metric values after every epoch to the shell apart from visualizing them in Visdom. The only change is to define `logger="shell"` instead of `"visdom"` in the input callbacks.

In [5]:
import logging

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)

# define perplexity callback for hold_out and test corpus
pl_holdout = PerplexityMetric(corpus=holdout_corpus, logger="shell")
pl_test = PerplexityMetric(corpus=test_corpus, logger="shell")

# define other remaining metrics available
ch_umass = CoherenceMetric(corpus=training_corpus, coherence="u_mass", logger="shell")
diff_kl = DiffMetric(distance="kullback_leibler", logger="shell")
convergence_jc = ConvergenceMetric(distance="jaccard", logger="shell")

callbacks = [pl_holdout, pl_test, ch_umass, diff_kl, convergence_jc]

# training LDA model
model = ldamodel.LdaModel(corpus=training_corpus, id2word=training_dictionary, passes=3, num_topics=5, callbacks=callbacks)

INFO:gensim.models.ldamodel:using symmetric alpha at 0.2
INFO:gensim.models.ldamodel:using symmetric eta at 0.0009950248756218905
INFO:gensim.models.ldamodel:using serial LDA version on this node
INFO:gensim.models.ldamodel:running online (multi-pass) LDA training, 5 topics, 3 passes over the supplied corpus of 25 documents, updating model once every 25 documents, evaluating perplexity every 25 documents, iterating 50x with a convergence threshold of 0.001000
INFO:gensim.models.ldamodel:-9.032 per-word bound, 523.4 perplexity estimate based on a held-out corpus of 25 documents with 2182 words
INFO:gensim.models.ldamodel:PROGRESS: pass 0, at document #25/25
INFO:gensim.models.ldamodel:topic #0 (0.200): 0.025*"the" + 0.022*"to" + 0.013*"in" + 0.011*"of" + 0.010*"and" + 0.007*"s" + 0.007*"a" + 0.005*"said" + 0.004*"he" + 0.004*"iraqi"
INFO:gensim.models.ldamodel:topic #1 (0.200): 0.054*"the" + 0.028*"to" + 0.023*"of" + 0.019*"and" + 0.017*"in" + 0.016*"a" + 0.013*"s" + 0.009*"that" + 0.00